In [16]:
%matplotlib inline
from __future__ import division

import pandas as pd
import utils

In [2]:
visitors, events, devices, url_categories = utils.load_data()

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [50]:
url_categories['stripped_url'] = url_categories.url.apply(lambda x: utils.clean_url(x, strip=True))

In [36]:
url_categories['clean_url'] = url_categories.url.apply(lambda x: utils.clean_url(x))
url_categories = url_categories.drop_duplicates(subset=['clean_url'])

In [79]:
url_categories = url_categories.drop_duplicates(subset=['clean_url'])

In [41]:
url_categories.to_csv('../data/url_categories_manual.csv')

In [33]:
event_urls = events.groupby('url').session_id.nunique().sort_values(ascending=False).reset_index()

In [34]:
event_urls['clean_url'] = event_urls.url.apply(lambda x: utils.clean_url(x))

In [51]:
event_urls['stripped_url'] = event_urls.url.apply(lambda x: utils.clean_url(x, strip=True))

In [80]:
event_urls = event_urls.rename(columns={'url':'event_url', 'clean_url':'event_clean_url'})
m = pd.merge(event_urls, url_categories, on='stripped_url', how='left')

In [81]:
bad_roots = url_categories.groupby('stripped_url').url.count().loc[lambda x: x>1].reset_index()
bad_roots.columns = ['stripped_url', 'cnt']
bad_roots

,stripped_url,cnt
0,eliquis,2
1,hcp/hcp,2
2,merck,3
3,merckclarifiesbiosimilars,2
4,merckconnect,11
5,merckvaccines,4
6,paradigmspine,13
7,podiatrytoday,2
8,revlimid/hcp,3


In [82]:
fix_me = pd.merge(m, bad_roots, on='stripped_url')
fix_me['match'] = fix_me.apply(lambda x: match_urls(x.clean_url, x.event_clean_url), axis=1)

In [119]:
m = m[~m.event_url.isin(fix_me.event_url)]

In [121]:
events_category = pd.merge(events, m, left_on='url', right_on='event_url')

In [125]:
events_cat = events_category.append(pd.merge(events, fix_me[fix_me.match==1],left_on='url', right_on='event_url' ))

In [124]:
len(events)

1790882

In [127]:
len(events_cat)/len(events)

0.9892042021752411

In [83]:
fix_me.loc[fix_me.clean_url==fix_me.event_clean_url, 'match'] = 1

In [109]:
def match_urls(url1, url2):
    if url1 == url2:
        return 1
    if '/' in url1 and '/' in url2:
        if url1.split('/')[1].split('/')[0] == url2.split('/')[1].split('/')[0]:
            return 1
        else:
            return 0
    else:
        return 0

In [110]:
fix_me['match'] = fix_me.apply(lambda x: match_urls(x.clean_url, x.event_clean_url), axis=1)

In [117]:
pd.merge(fix_me[fix_me.match==1], fix_me[fix_me.match==0], on='event_url')

,event_url,session_id_x,event_clean_url_x,stripped_url_x,url_x,site_category_x,site_sub_category_x,disease_category_x,disease_x,pharma_firm_x,...,url_y,site_category_y,site_sub_category_y,disease_category_y,disease_y,pharma_firm_y,clean_url_y,cnt_y,match_y,url_match_y
0,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/belsomra/,Pharma,Branded HCP,Psychiatry,Insomnia,Merck,merckconnect.com/belsomra,11,0,0
1,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/dificid/,Pharma,Branded HCP,Gastroenterology,Clostridium difficile�associated diarrhea (CDAD).,Merck,merckconnect.com/dificid,11,0,0
2,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/dulera/,Pharma,Branded HCP,"Allergy & Immunology, \rOtolaryngology, Pulmo...",Asthma,Merck,merckconnect.com/dulera,11,0,0
3,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/isentress/,Pharma,Branded HCP,Infectious Disease,HIV,Merck,merckconnect.com/isentress,11,0,0
4,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/janumet/,Pharma,Branded HCP,Endocrinology,type 2 diabetes,Merck,merckconnect.com/janumet,11,0,0
5,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/janumetxr/,Pharma,Branded HCP,Endocrinology,type 2 diabetes,Merck,merckconnect.com/janumetxr,11,0,0
6,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/januvia/,Pharma,Branded HCP,Endocrinology,type 2 diabetes,Merck,merckconnect.com/januvia,11,0,0
7,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/nexplanon/,Pharma,Branded HCP,Obstetrics & Gynecology,Birth Control,Merck,merckconnect.com/nexplanon,11,0,0
8,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/noxafil/,Pharma,Branded HCP,Oncology,invasive�Aspergillus�and�Candida�infections,Merck,merckconnect.com/noxafil,11,0,0
9,www.merckconnect.com/,282,merckconnect.com,merckconnect,www.merckconnect.com,Pharma,Branded HCP,Multiple,Multiple,Merck,...,https://www.merckconnect.com/zepatier/,Pharma,Branded HCP,Infectious Disease,chronic hepatitis C virus (HCV),Merck,merckconnect.com/zepatier,11,0,0


In [113]:
fix_me[(fix_me.url_match==0)&(fix_me.site_category=='Pharma')][['clean_url', 'event_clean_url']].sample(10)

,clean_url,event_clean_url
2547,merckconnect.com/januvia,merckconnect.com/noxafil/overview.html?utm_sou...
4472,merckconnect.com/januvia,merckconnect.com/bridion/dosing.html?&gclid=ci...
4074,merckconnect.com/janumet,merckconnect.com/zinplava/overview.html?&gclid...
2249,merckconnect.com/janumetxr,merckconnect.com/bridion/efficacy-reversal-in-...
6834,merckconnect.com/isentress,merckconnect.com/belsomra/overview.html?&gclid...
1864,merckconnect.com/janumetxr,merckconnect.com/healthcaretopics/medication-a...
2749,merckconnect.com,merckconnect.com/dulera/key-data-regarding-the...
8189,merckconnect.com/janumetxr,merckconnect.com/januvia/professional-resource...
2285,merckconnect.com/noxafil,merckconnect.com/januvia/fpg-ppg-reductions.html
1574,merckconnect.com/dificid,merckconnect.com/januvia/formulary-coverage.ht...


In [115]:
'merckconnect.com/noxafil/overview.html?'.split('/')[1].split('/')[0]

'noxafil'

In [89]:
fix_me[fix_me.apply(lambda x: x.event_clean_url in x.clean_url, axis=1)][['event_clean_url', 'clean_url', 'match']]

,event_clean_url,clean_url,match
33,merckconnect.com,merckconnect.com/belsomra,NaN
34,merckconnect.com,merckconnect.com/dificid,NaN
35,merckconnect.com,merckconnect.com/dulera,NaN
36,merckconnect.com,merckconnect.com/isentress,NaN
37,merckconnect.com,merckconnect.com/janumet,NaN
38,merckconnect.com,merckconnect.com/janumetxr,NaN
39,merckconnect.com,merckconnect.com/januvia,NaN
40,merckconnect.com,merckconnect.com/nexplanon,NaN
41,merckconnect.com,merckconnect.com/noxafil,NaN
42,merckconnect.com,merckconnect.com/zepatier,NaN


In [38]:
sum(m[m.site_category.isnull()].session_id)/sum(m.session_id)

0.9372018971572913

In [17]:
sum(event_urls[event_urls.site_category.isnull()].session_id)/sum(event_urls.session_id)
# less than 2% don't match the list, cool cool that's fine

0.01695353739597159

In [32]:
event_urls

,url_x,session_id,stripped_url,url_y,site_category,site_sub_category,disease_category,disease,pharma_firm
0,reachmd.com/profiles/subscriptions/control/com...,28638,reachmd,www.ReachMD.com,Multi-Channel Marketing,Multi Topic,Multiple,NaN,NaN
1,www.medilexicon.com/,21318,medilexicon,www.medilexicon.com,Publisher,Multi Topic,Multiple,NaN,NaN
2,www.medicalnewstoday.com/,20530,medicalnewstoday,www.medicalnewstoday.com,Publisher,Multi Topic,Multiple,NaN,NaN
3,www.globalrph.com/,16058,globalrph,globalrph.com,Reference Tool,Multi Topic,Multiple,NaN,NaN
4,www.globalrph.com/medcalcs.htm,14530,globalrph,globalrph.com,Reference Tool,Multi Topic,Multiple,NaN,NaN
5,www.consultant360.com/home,10771,consultant360,www.consultant360.com,Publisher,Multi Topic,Multiple,NaN,NaN
6,www.medilexicon.com/abbreviations,10383,medilexicon,www.medilexicon.com,Publisher,Multi Topic,Multiple,NaN,NaN
7,www.globalrph.com/multiple_crcl.htm,10320,globalrph,globalrph.com,Reference Tool,Multi Topic,Multiple,NaN,NaN
8,www.globalrph.com/multiple_crcl.cgi,9324,globalrph,globalrph.com,Reference Tool,Multi Topic,Multiple,NaN,NaN
9,www.consultant360.com/topic/fda-alerts,8124,consultant360,www.consultant360.com,Publisher,Multi Topic,Multiple,NaN,NaN


In [31]:
event_urls.groupby(['url_x', 'stripped_url', 'url_y']).session_id.count().loc[lambda x: x>1].reset_index().sort_values('session_id', ascending=False)

,url_x,stripped_url,url_y,session_id


In [ ]:
'revlimid/hcp':mm/mcl/mds

In [29]:
url_categories[url_categories.stripped_url=='paradigmspine']

,url,site_category,site_sub_category,disease_category,disease,pharma_firm,stripped_url
0,http://www.paradigmspine.com/content/are-you-a...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
1,http://www.paradigmspine.com/content/back-spin...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
2,http://www.paradigmspine.com/content/coflex-study,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
3,http://www.paradigmspine.com/content/degenerat...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
4,http://www.paradigmspine.com/content/degenerat...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
5,http://www.paradigmspine.com/content/glossary,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
6,http://www.paradigmspine.com/content/patients-...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
7,http://www.paradigmspine.com/content/patient-t...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
8,http://www.paradigmspine.com/content/spinal-st...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine
9,http://www.paradigmspine.com/content/spondylol...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine


In [28]:
event_urls[event_urls.stripped_url=='paradigmspine']

,url_x,session_id,stripped_url,url_y,site_category,site_sub_category,disease_category,disease,pharma_firm
2788,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/are-you-a...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2789,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/back-spin...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2790,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/coflex-study,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2791,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/degenerat...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2792,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/degenerat...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2793,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/glossary,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2794,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/patients-...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2795,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/patient-t...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2796,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/spinal-st...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine
2797,www.paradigmspine.com/content/coflex-interlami...,69,paradigmspine,http://www.paradigmspine.com/content/spondylol...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine


In [33]:
url_categories[url_categories.site_sub_category=='Branded HCP'][['url', '']]

,url,site_category,site_sub_category,disease_category,disease,pharma_firm,stripped_url
15,http://www.adempas-us.com/hcp/,Pharma,Branded HCP,"Cardiology, Pulmonology\rMultiple","Thromboembolic Pulmonary Hypertension, Pulmona...",Bayer,adempas-us
16,http://www.herceptin.com/hcp/,Pharma,Branded HCP,Oncology,"Metastatic Breast Cancer, Metastatic Gastric C...",Genentech,herceptin
18,http://www.kadcyla.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,Genentech,kadcyla
19,http://www.lenvima.com/hcp/,Pharma,Branded HCP,Oncology,"Thyroid Cancer, advanced renal cell carcinoma ...",Eisai,lenvima
20,http://www.merck.com/product/home.html,Pharma,Branded HCP,Multiple,Multiple,Merck,merck
21,http://www.merck.com/product/usa/pi_circulars/...,Pharma,Branded HCP,"Allergy & Immunology, Otolaryngology, Pulmonol...",Asthma,Merck,merck
22,http://www.merck.com/product/usa/pi_circulars/...,Pharma,Branded HCP,"Allergy & Immunology, Otolaryngology, Pulmonol...",Asthma,Merck,merck
23,http://www.perjeta.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,Genentech,perjeta
24,http://www.pomalyst.com/hcp/,Pharma,Branded HCP,Oncology,multiple myeloma,Celgene,pomalyst
25,http://www.rituxan.com/hem/hcp,Pharma,Branded HCP,"Oncology, Rheumatology\rMultiple","non-Hodgkin's lymphoma, �chronic lymphocytic l...",Genentech,rituxan


In [32]:
url_categories[url_categories.stripped_url=='keytruda']

,url,site_category,site_sub_category,disease_category,disease,pharma_firm,stripped_url
37,https://www.keytruda.com/hcp/,Pharma,Branded HCP,Oncology,"melanoma, non�small cell lung cancer (NSCLC), ...",Merck,keytruda
96,www.keytruda.com,Pharma,Branded Consumer,Oncology,"melanoma, non�small cell lung cancer (NSCLC), ...",Merck,keytruda


In [54]:
re.split('[^a-zA-Z]', 'a.a88..')

['a', 'a', '', '', '', '']

In [57]:
re.split('[^a-zA-Z]', 'https://www.keytruda.com/hcp/')

['https', '', '', 'www', 'keytruda', 'com', 'hcp', '']

In [45]:
hcp.findall('www.keyhcptruda.com')

['hcp']

In [46]:
event_urls = good_events.url.value_counts().reset_index()
event_urls.columns = ['url', 'cnt']

In [51]:
event_urls['clean_url'] = event_urls.url.apply(lambda x: clean_url(x))

In [53]:
pd.merge(pharma, event_urls, on='clean_url')

,url_x,site_category,site_sub_category,disease_category,disease,clean_url,url_y,cnt
0,http://www.adempas-us.com/hcp/,Pharma,Branded HCP,"Cardiology, Pulmonology\nMultiple","Thromboembolic Pulmonary Hypertension, Pulmona...",adempas-us.com/hcp,www.adempas-us.com/hcp/,24
1,http://www.herceptin.com/hcp/,Pharma,Branded HCP,Oncology,"Metastatic Breast Cancer, Metastatic Gastric C...",herceptin.com/hcp,www.herceptin.com/hcp/,34
2,http://www.journeyforcontrol.com/diabetes-educ...,Pharma,Unbranded HCP,Endocrinology,Diabetes,journeyforcontrol.com/diabetes-educator,www.journeyforcontrol.com/diabetes-educator/,294
3,http://www.kadcyla.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,kadcyla.com/hcp,www.kadcyla.com/hcp/,13
4,http://www.lenvima.com/hcp/,Pharma,Branded HCP,Oncology,"Thyroid Cancer, advanced renal cell carcinoma ...",lenvima.com/hcp,www.lenvima.com/hcp/,53
5,http://www.perjeta.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,perjeta.com/hcp,www.perjeta.com/hcp/,76
6,http://www.pomalyst.com/,Pharma,Branded HCP,Oncology,multiple myeloma,pomalyst.com,www.pomalyst.com/,309
7,http://www.rituxan.com/hem/hcp,Pharma,Branded HCP,"Oncology, Rheumatology\nMultiple","non-Hodgkin's lymphoma, chronic lymphocytic l...",rituxan.com/hem/hcp,www.rituxan.com/hem/hcp,32
8,http://www.rituxan.com/hem/hcp,Pharma,Branded HCP,"Oncology, Rheumatology\nMultiple","non-Hodgkin's lymphoma, chronic lymphocytic l...",rituxan.com/hem/hcp,www.rituxan.com/hem/hcp/,1
9,http://www.tarceva.com/hcp/,Pharma,Branded HCP,Oncology,"non-small cell lung cancer (NSCLC), advanced-s...",tarceva.com/hcp,www.tarceva.com/hcp/,10


In [57]:
event_urls['clean_url_root'] = event_urls.url.apply(lambda x: clean_url(x, extract_root=True))

In [ ]:
pd.merge(pharma, event_urls, on='clean_url')

In [62]:
good_events['url_root'] = good_events.url.apply(lambda x: clean_url(x, extract_root=True))

In [64]:
pharma = pharma.rename(columns={'clean_url_root':'url_root'})

In [127]:
pharma_visits =pd.merge(pharma, good_events, on='url_root')

In [128]:
pharma_visits['authorized_visit'] = pharma_visits.identity_type.apply(lambda x: 1 if x=='AUT' else 0)

In [129]:
pharma_ratio = pharma_visits.groupby('url_root').authorized_visit.agg(['sum', 'count']).reset_index().rename(columns={'sum':'authorized_views', 'count':'total_views'})

In [130]:
from __future__ import division
pharma_ratio['pct_authorized'] = pharma_ratio.apply(lambda x: x['authorized_views']/x['total_views'], axis=1)

In [132]:
pharma = pd.merge(pharma, pharma_ratio[['url_root', 'pct_authorized', 'authorized_views', 'total_views']], on='url_root')

In [133]:
pharma['pharma_id'] = range(len(pharma))

In [135]:
p = pharma[['pharma_id', 'disease_category', 'disease', 'pct_authorized', 'authorized_views', 'total_views']]

In [136]:
t1 = pd.merge(p, p, on ='disease_category')

In [137]:
t1 = t1[t1.pharma_id_x!=t1.pharma_id_y]

In [144]:
t1.groupby('pharma_id_x')[['authorized_views_y', 'total_views_y']].agg(['sum', 'count'])

authorized_views_y       total_views_y      
                           sum count           sum count
pharma_id_x                                             
0                          220     3           298     3
1                          220     3           298     3
2                        64479    33         93915    33
3                        64479    33         93915    33
4                       255570     7        335597     7
5                       255570     7        335597     7
6                        64775    33         94387    33
7                        64775    33         94387    33
8                        63353    33         92543    33
9                        63353    33         92543    33
10                       93538     8        123913     8
11                       93538     8        123913     8
12                       94476     8        125413     8
13                         692     1           912     1
14                         692     1           912     1
15                       64741    33         94327    33
16                       64741    33         94327    33
17                       83869     4        109815     4
18                       83869     4        109815     4
19                       94970     8        125995     8
20                       94970     8        125995     8
21                       64933    33         94537    33
22                       64933    33         94537    33
23                       95114     8        126261     8
24                       65051    33         94749    33
25                        1070     4          1486     4
26                        1070     4          1486     4
27                        1070     4          1486     4
28                       65023    33         94737    33
29                       65023    33         94737    33
...                        ...   ...           ...   ...
80                        3824     2          5224     2
81                      169744     9        222458     9
82                         449     4           627     4
83                        1251     4          1672     4
84                       85413     2        111983     2
85                       95066     8        126193     8
86                       90072     2        119167     2
87                       65090    33         94808    33
88                         863     3          1188     3
89                        2536     3          3518     3
90                       65091    33         94805    33
91                      168935     9        221365     9
92                       87390     8        115233     8
93                       57327    33         83721    33
94                       57327    33         83721    33
95                       57327    33         83721    33
96                        1368     1          1720     1
97                      169363     9        221961     9
98                       65086    33         94802    33
99                      257342     7        338106     7
100                       1245     4          1650     4
101                      65090    33         94808    33
102                      65054    33         94741    33
103                        968     4          1270     4
104                       1271     4          1693     4
105                      87742     2        115152     2
106                      64831    33         94455    33
107                      64782    33         94341    33
108                     169051     9        221566     9
109                     168477     9        220623     9

[106 rows x 4 columns]

In [145]:
pharma.sample()

,url,site_category,site_sub_category,disease_category,disease,url_root,pct_authorized,authorized_views,total_views,pharma_id
21,https://www.alecensa.com/hcp.html,Pharma,Branded HCP,Oncology,"anaplastic lymphoma kinase (ALK)-positive, met...",alecensa,0.580882,158,272,21


In [147]:
client_id = 21

In [149]:
pharma[pharma.pharma_id==21].pct_authorized

21    0.580882
Name: pct_authorized, dtype: float64

In [ ]:
def deep_dive()